In [1]:
import datetime
import os
import argparse
import torch

from rlpyt.runners.minibatch_rl import MinibatchRlEval, MinibatchRl
from rlpyt.samplers.serial.sampler import SerialSampler
from rlpyt.utils.logging.context import logger_context
from rlpyt.envs.atari.atari_env import AtariTrajInfo

from dreamer_agent import AtariDreamerAgent
from algorithm import Dreamer
from envs.atari import AtariEnv
from envs.wrapper import make_wapper
from envs.one_hot import OneHotAction
from envs.time_limit import TimeLimit




def build_and_train(
    log_dir,
    game="pong",
    run_ID=0,
    cuda_idx=None,
    eval=False,
    save_model="last",
    load_model_path=None,
):
    params = torch.load(load_model_path) if load_model_path else {}
    agent_state_dict = params.get("agent_state_dict")
    optimizer_state_dict = params.get("optimizer_state_dict")

    action_repeat = 2
    env_kwargs = dict(
        name=game,
        action_repeat=action_repeat,
        size=(64, 64),
        grayscale=False,
        life_done=True,
        sticky_actions=False,
    )
    factory_method = make_wapper(
        AtariEnv,
        [OneHotAction, TimeLimit],
        [dict(), dict(duration=1000 / action_repeat)],
    )
    sampler = SerialSampler(
        EnvCls=factory_method,
        TrajInfoCls=AtariTrajInfo,  # default traj info + GameScore
        env_kwargs=env_kwargs,
        eval_env_kwargs=env_kwargs,
        batch_T=1,
        batch_B=1,
        max_decorrelation_steps=0,
        eval_n_envs=10,
        eval_max_steps=int(10e3),
        eval_max_trajectories=5,
    )
    algo = Dreamer(
        horizon=10,
        kl_scale=0.1,
        use_pcont=True,
        initial_optim_state_dict=optimizer_state_dict,
    )
    agent = AtariDreamerAgent(
        train_noise=0.4,
        eval_noise=0,
        expl_type="epsilon_greedy",
        expl_min=0.1,
        expl_decay=2000 / 0.3,
        initial_model_state_dict=agent_state_dict,
        model_kwargs=dict(use_pcont=True),
    )
    runner_cls = MinibatchRlEval if eval else MinibatchRl
    runner = runner_cls(
        algo=algo,
        agent=agent,
        sampler=sampler,
        n_steps=5e6,
        log_interval_steps=1e3,
        affinity=dict(cuda_idx=cuda_idx),
    )
    config = dict(game=game)
    name = "dreamer_" + game
    with logger_context(
        log_dir,
        run_ID,
        name,
        config,
        snapshot_mode=save_model,
        override_prefix=True,
        use_summary_writer=True,
    ):
        runner.train()



2024-01-10 17:57:36.101465: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-10 17:57:36.958596: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
log_dir = os.path.abspath('/home/eddy/Projects/RL_project/logs_atari')

build_and_train(
    log_dir ,
    game="assault",
    run_ID=5,
    cuda_idx=0,
    eval=False,
    save_model="last",
    load_model_path=False,
)

2024-01-10 17:57:37.909526  | dreamer_assault_5 Runner  master CPU affinity: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11].
2024-01-10 17:57:37.910422  | dreamer_assault_5 Runner  master Torch threads: 6.
using seed 5202
2024-01-10 17:57:39.789140  | dreamer_assault_5 Sampler decorrelating envs, max steps: 0
2024-01-10 17:57:39.789930  | dreamer_assault_5 Serial Sampler initialized.
2024-01-10 17:57:39.790620  | dreamer_assault_5 Running 5000000 iterations of minibatch RL.
2024-01-10 17:57:40.727269  | dreamer_assault_5 Initialized agent model on device: cuda:0.
DAI CHE SIAMO VICINIIIIIIII
size passato a SequenceNStepReturnBuffer = 300000
2024-01-10 17:57:40.728858  | dreamer_assault_5 Optimizing over 1000 iterations.


/home/eddy/.local/lib/python3.10/site-packages/torch/optim/adam.py:33: UserWarning: optimizer contains a parameter group with duplicate parameters; in future, this will cause an error; see github.com/pytorch/pytorch/issues/40967 for more information
  super().__init__(params, defaults)


2024-01-10 17:57:46.207644  | dreamer_assault_5 itr #999 saving snapshot...
2024-01-10 17:57:46.265182  | dreamer_assault_5 itr #999 saved
2024-01-10 17:57:46.273708  | -----------------------------  ----------
2024-01-10 17:57:46.274855  | Diagnostics/NewCompletedTrajs     3
2024-01-10 17:57:46.275243  | Diagnostics/StepsInTrajWindow   862
2024-01-10 17:57:46.275704  | Diagnostics/Iteration           999
2024-01-10 17:57:46.276111  | Diagnostics/CumTime (s)           5.53667
2024-01-10 17:57:46.276536  | Diagnostics/CumSteps           1000
2024-01-10 17:57:46.277053  | Diagnostics/CumCompletedTrajs     3
2024-01-10 17:57:46.277520  | Diagnostics/CumUpdates            0
2024-01-10 17:57:46.278182  | Diagnostics/StepsPerSecond      180.614
2024-01-10 17:57:46.280022  | Diagnostics/UpdatesPerSecond      0
2024-01-10 17:57:46.280737  | Diagnostics/ReplayRatio           0
2024-01-10 17:57:46.281684  | Diagnostics/CumReplayRatio        0
2024-01-10 17:57:46.282306  | Length/Average         

Imagination: 100%|██████████| 100/100 [01:59<00:00,  1.20s/it]


2024-01-10 18:00:08.692312  | dreamer_assault_5 itr #5999 saving snapshot...
2024-01-10 18:00:08.869601  | dreamer_assault_5 itr #5999 saved
2024-01-10 18:00:08.892172  | -----------------------------  ---------------
2024-01-10 18:00:08.893247  | Diagnostics/NewCompletedTrajs      4
2024-01-10 18:00:08.894111  | Diagnostics/StepsInTrajWindow   5772
2024-01-10 18:00:08.895661  | Diagnostics/Iteration           5999
2024-01-10 18:00:08.896714  | Diagnostics/CumTime (s)          148.142
2024-01-10 18:00:08.897618  | Diagnostics/CumSteps            6000
2024-01-10 18:00:08.898561  | Diagnostics/CumCompletedTrajs     20
2024-01-10 18:00:08.900070  | Diagnostics/CumUpdates             0
2024-01-10 18:00:08.901136  | Diagnostics/StepsPerSecond         8.07668
2024-01-10 18:00:08.902040  | Diagnostics/UpdatesPerSecond       0
2024-01-10 18:00:08.902948  | Diagnostics/ReplayRatio            0
2024-01-10 18:00:08.904299  | Diagnostics/CumReplayRatio         0
2024-01-10 18:00:08.905097  | Lengt

...

2024-01-10 18:46:05.042478  | dreamer_assault_5 itr #26999 saving snapshot...
2024-01-10 18:46:05.158764  | dreamer_assault_5 itr #26999 saved
2024-01-10 18:46:05.167518  | -----------------------------  ----------------
2024-01-10 18:46:05.168126  | Diagnostics/NewCompletedTrajs       4
2024-01-10 18:46:05.168613  | Diagnostics/StepsInTrajWindow   26912
2024-01-10 18:46:05.168996  | Diagnostics/Iteration           26999
2024-01-10 18:46:05.169415  | Diagnostics/CumTime (s)          2904.43
2024-01-10 18:46:05.169850  | Diagnostics/CumSteps            27000
2024-01-10 18:46:05.170435  | Diagnostics/CumCompletedTrajs      99
2024-01-10 18:46:05.170896  | Diagnostics/CumUpdates              0
2024-01-10 18:46:05.171488  | Diagnostics/StepsPerSecond          7.6702
2024-01-10 18:46:05.171966  | Diagnostics/UpdatesPerSecond        0
2024-01-10 18:46:05.172336  | Diagnostics/ReplayRatio             0
2024-01-10 18:46:05.172698  | Diagnostics/CumReplayRatio          0
2024-01-10 18:46:05.173

Imagination: 100%|██████████| 100/100 [02:09<00:00,  1.30s/it]


2024-01-10 18:48:19.480114  | dreamer_assault_5 itr #27999 saving snapshot...
2024-01-10 18:48:19.658034  | dreamer_assault_5 itr #27999 saved
2024-01-10 18:48:19.666425  | -----------------------------  ----------------
2024-01-10 18:48:19.667125  | Diagnostics/NewCompletedTrajs       3
2024-01-10 18:48:19.667445  | Diagnostics/StepsInTrajWindow   27204
2024-01-10 18:48:19.667766  | Diagnostics/Iteration           27999
2024-01-10 18:48:19.668368  | Diagnostics/CumTime (s)          3038.93
2024-01-10 18:48:19.668765  | Diagnostics/CumSteps            28000
2024-01-10 18:48:19.669354  | Diagnostics/CumCompletedTrajs     102
2024-01-10 18:48:19.669706  | Diagnostics/CumUpdates              0
2024-01-10 18:48:19.670125  | Diagnostics/StepsPerSecond          7.43498
2024-01-10 18:48:19.670738  | Diagnostics/UpdatesPerSecond        0
2024-01-10 18:48:19.671778  | Diagnostics/ReplayRatio             0
2024-01-10 18:48:19.672289  | Diagnostics/CumReplayRatio          0
2024-01-10 18:48:19.67

Imagination:  36%|███▌      | 36/100 [00:47<01:24,  1.33s/it]


KeyboardInterrupt: 